# Subduction Zone Benchmark

Author: Cian Wilson

**Under Construction!**

## Parallel Scaling

In [the previous notebook](./3.4b_sz_benchmark.ipynb) we tested that the error in our implementation of a steady-state thermal convection problem in two-dimensions converged towards the published benchmark value as the resolution scale (`resscale`) decreased (the number of elements increased).  We also wish to test for parallel scaling of this problem, assessing if the simulation wall time decreases as the number of processors used to solve it is increases.

Here we perform strong scaling tests on our functions `solve_benchmark_case1` and `solve_benchmark_case2` from [the previous notebook](./3.4b_sz_benchmark.ipynb).

### Preamble

We start by loading all the modules we will require.

In [ ]:
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
path = os.path.join(basedir, os.path.pardir, os.path.pardir, 'python')
sys.path.append(path)
import utils.ipp
import matplotlib.pyplot as pl
import numpy as np
import pathlib
output_folder = pathlib.Path(os.path.join(basedir, "output"))
output_folder.mkdir(exist_ok=True, parents=True)

### Implementation

We perform the strong parallel scaling test using a utility function (from `python/utils/ipp.py`) that loops over a list of the number of processors calling our function for a given number of elements, `ne`, and pressure and temperature polynomial orders `pp` and `pT`.  It runs our function `solve_blankenbach` a specified `number` of times and evaluates and returns the time taken for each of a number of requested `steps`.

In [ ]:
# the list of the number of processors we will use
nprocs_scale = [1, 2, 3, 4, 6, 8,]

# the number of elements to solve the problem on
resscale_scale = 0.5

# perform the calculation a set number of times
number = 10

# We are interested in the time to create the mesh,
# declare the functions, assemble the problem and solve it.
# From our implementation in `solve_poisson_2d` it is also
# possible to request the time to declare the Dirichlet and
# Neumann boundary conditions and the forms.
steps = [
          'Assemble Temperature', 'Assemble Stokes',
          'Solve Temperature', 'Solve Stokes'
         ]


#### Case 1

In [ ]:
# declare a dictionary to store the times each step takes
maxtimes_1 = {}

In [ ]:
maxtimes_1['Direct Stokes'] = utils.ipp.profile_parallel(nprocs_scale, steps, path, 
                                                         'sz_problems.sz_benchmark', 'solve_benchmark_case1', 
                                                        resscale_scale, number=number,
                                                        output_filename=output_folder / 'sz_benchmark_scaling_direct_1.png')

In [ ]:
# the list of the number of processors to test the convergence on
nprocs_conv = [1, 4,]

# List of resolutions to try
resscales_conv = [0.5,]

In [ ]:
diagnostics = []
for resscalel in resscales_conv:
    diagnostics.append(utils.ipp.run_parallel(nprocs_conv, path, 
                                           'sz_problems.sz_benchmark', 
                                           'solve_benchmark_case1', 
                                           resscalel))

print('')
print('{:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}'.format('nprocs', 'resscale', 'T_ndof', 'T_{200,-100}', 'Tbar_s', 'Tbar_w', 'Vrmsw'))
for resscalel, diag_all in zip(resscales_conv, diagnostics):
    for i, nproc in enumerate(nprocs_conv):
        print('{:<12d} {:<12.4g} {:<12d} {:<12.4f} {:<12.4f} {:<12.4f} {:<12.4f}'.format(nproc, resscalel, *diag_all[i].values()))
         # check error (compared to the serial case) is less than 0.1%
        if i > 0: 
            for k, v in diag_all[i].items():
                assert(abs(v - diag_all[0][k])/abs(diag_all[0][k]) < 1.e-3)

In [ ]:
petsc_options_s = {'ksp_type':'minres', 
                   'ksp_rtol' : 5.e-9,
                   'pc_type':'fieldsplit', 
                   'pc_fieldsplit_type': 'additive',
                   'fieldsplit_v_ksp_type':'preonly',
                   'fieldsplit_v_pc_type':'gamg',
                   'fieldsplit_p_ksp_type':'preonly',
                   'fieldsplit_p_pc_type':'jacobi'}

maxtimes_1['Iterative Stokes'] = utils.ipp.profile_parallel(nprocs_scale, steps, path, 
                                                            'sz_problems.sz_benchmark', 'solve_benchmark_case1', 
                                                            resscale_scale, petsc_options_s=petsc_options_s, number=number,
                                                            output_filename=output_folder / 'sz_benchmark_scaling_iterative_1.png')

In [ ]:
diagnostics = []
for resscalel in resscales_conv:
    diagnostics.append(utils.ipp.run_parallel(nprocs_conv, path, 
                                           'sz_problems.sz_benchmark', 
                                           'solve_benchmark_case1', 
                                           resscalel, petsc_options_s=petsc_options_s))

print('')
print('{:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}'.format('nprocs', 'resscale', 'T_ndof', 'T_{200,-100}', 'Tbar_s', 'Tbar_w', 'Vrmsw'))
for resscalel, diag_all in zip(resscales_conv, diagnostics):
    for i, nproc in enumerate(nprocs_conv):
        print('{:<12d} {:<12.4g} {:<12d} {:<12.4f} {:<12.4f} {:<12.4f} {:<12.4f}'.format(nproc, resscalel, *diag_all[i].values()))
         # check error (compared to the serial case) is less than 0.1%
        if i > 0: 
            for k, v in diag_all[i].items():
                assert(abs(v - diag_all[0][k])/abs(diag_all[0][k]) < 1.e-3)

In [ ]:
# choose which steps to compare
compare_steps = ['Assemble Stokes', 'Solve Stokes']

# set up a figure for plotting
fig, axs = pl.subplots(nrows=len(compare_steps), figsize=[6.4,4.8*len(compare_steps)], sharex=True)
if len(compare_steps) == 1: axs = [axs]
for i, step in enumerate(compare_steps):
    s = steps.index(step)
    for name, lmaxtimes in maxtimes_1.items():
        axs[i].plot(nprocs_scale, [t[s] for t in lmaxtimes], 'o-', label=name)
    axs[i].set_title(step)
    axs[i].legend()
    axs[i].set_ylabel('wall time (s)')
    axs[i].grid()
axs[-1].set_xlabel('number processors')
# save the figure
fig.savefig(output_folder / 'sz_benchmark_scaling_comparison_1.png')

#### Case 2

In [ ]:
# declare a dictionary to store the times each step takes
maxtimes_2 = {}

In [ ]:
maxtimes_2['Direct Stokes'] = utils.ipp.profile_parallel(nprocs_scale, steps, path, 
                                                         'sz_problems.sz_benchmark', 'solve_benchmark_case2', 
                                                        resscale_scale, number=number,
                                                        output_filename=output_folder / 'sz_benchmark_scaling_direct_2.png')

In [ ]:
diagnostics = []
for resscalel in resscales_conv:
    diagnostics.append(utils.ipp.run_parallel(nprocs_conv, path, 
                                           'sz_problems.sz_benchmark', 
                                           'solve_benchmark_case2', 
                                           resscalel))

print('')
print('{:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}'.format('nprocs', 'resscale', 'T_ndof', 'T_{200,-100}', 'Tbar_s', 'Tbar_w', 'Vrmsw'))
for resscalel, diag_all in zip(resscales_conv, diagnostics):
    for i, nproc in enumerate(nprocs_conv):
        print('{:<12d} {:<12.4g} {:<12d} {:<12.4f} {:<12.4f} {:<12.4f} {:<12.4f}'.format(nproc, resscalel, *diag_all[i].values()))
         # check error (compared to the serial case) is less than 0.1%
        if i > 0: 
            for k, v in diag_all[i].items():
                assert(abs(v - diag_all[0][k])/abs(diag_all[0][k]) < 1.e-3)

In [ ]:
petsc_options_s = {'ksp_type':'minres', 
                   'ksp_rtol' : 5.e-9,
                   'pc_type':'fieldsplit', 
                   'pc_fieldsplit_type': 'additive',
                   'fieldsplit_v_ksp_type':'preonly',
                   'fieldsplit_v_pc_type':'gamg',
                   'fieldsplit_p_ksp_type':'preonly',
                   'fieldsplit_p_pc_type':'jacobi'}

maxtimes_2['Iterative Stokes'] = utils.ipp.profile_parallel(nprocs_scale, steps, path, 
                                                            'sz_problems.sz_benchmark', 'solve_benchmark_case2', 
                                                            resscale_scale, petsc_options_s=petsc_options_s, number=number,
                                                            output_filename=output_folder / 'sz_benchmark_scaling_iterative_2.png')

In [ ]:
diagnostics = []
for resscalel in resscales_conv:
    diagnostics.append(utils.ipp.run_parallel(nprocs_conv, path, 
                                           'sz_problems.sz_benchmark', 
                                           'solve_benchmark_case2', 
                                           resscalel, petsc_options_s=petsc_options_s))

print('')
print('{:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}'.format('nprocs', 'resscale', 'T_ndof', 'T_{200,-100}', 'Tbar_s', 'Tbar_w', 'Vrmsw'))
for resscalel, diag_all in zip(resscales_conv, diagnostics):
    for i, nproc in enumerate(nprocs_conv):
        print('{:<12d} {:<12.4g} {:<12d} {:<12.4f} {:<12.4f} {:<12.4f} {:<12.4f}'.format(nproc, resscalel, *diag_all[i].values()))
         # check error (compared to the serial case) is less than 0.1%
        if i > 0: 
            for k, v in diag_all[i].items():
                assert(abs(v - diag_all[0][k])/abs(diag_all[0][k]) < 1.e-3)

In [ ]:
# choose which steps to compare
compare_steps = ['Assemble Stokes', 'Solve Stokes']

# set up a figure for plotting
fig, axs = pl.subplots(nrows=len(compare_steps), figsize=[6.4,4.8*len(compare_steps)], sharex=True)
if len(compare_steps) == 1: axs = [axs]
for i, step in enumerate(compare_steps):
    s = steps.index(step)
    for name, lmaxtimes in maxtimes_2.items():
        axs[i].plot(nprocs_scale, [t[s] for t in lmaxtimes], 'o-', label=name)
    axs[i].set_title(step)
    axs[i].legend()
    axs[i].set_ylabel('wall time (s)')
    axs[i].grid()
axs[-1].set_xlabel('number processors')
# save the figure
fig.savefig(output_folder / 'sz_benchmark_scaling_comparison_2.png')